In [1]:
# Compare Active Learning Strategies - Regression (with CV + Multiple Trials)
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

SAVE_DIR = os.path.join('..', 'report', 'figures')
DATA_DIR = os.path.join('..', 'data')
os.makedirs(SAVE_DIR, exist_ok=True)

# Load results (these should be generated by the updated notebooks)
try:
    with open(os.path.join(DATA_DIR, 'reg_uncertainty_results.json'), 'r') as f:
        unc = json.load(f)
    with open(os.path.join(DATA_DIR, 'reg_sensitivity_results.json'), 'r') as f:
        sen = json.load(f)
    with open(os.path.join(DATA_DIR, 'passive_reg_best.json'), 'r') as f:
        pas = json.load(f)
except FileNotFoundError as e:
    print(f"Results file not found: {e}")
    print("Please run the updated exploration notebooks first to generate results with CV + multiple trials")
    exit()

DATASETS = ['diabetes', 'linnerud', 'california']
METRICS = ['rmse', 'mae', 'r2']


In [2]:
# Compare all datasets
for dataset in DATASETS:
    print(f"\n=== Comparing strategies for {dataset} ===")
    
    # Get budgets and methods
    budgets = sorted([int(b) for b in next(iter(unc[dataset].values())).keys()])
    methods = list(unc[dataset].keys())
    
    for metric in METRICS:
        plt.figure(figsize=(10, 6))
        
        # Plot uncertainty methods
        for m in methods:
            means = [unc[dataset][m][str(b)][f'{metric}_mean'] for b in budgets]
            stds = [unc[dataset][m][str(b)][f'{metric}_std'] for b in budgets]
            plt.plot(budgets, means, marker='o', label=f'uncertainty_{m}', linewidth=2)
            plt.fill_between(budgets, np.array(means)-np.array(stds), np.array(means)+np.array(stds), alpha=0.2)
        
        # Plot sensitivity method
        if dataset in sen:
            means = [sen[dataset][str(b)][f'{metric}_mean'] for b in budgets]
            stds = [sen[dataset][str(b)][f'{metric}_std'] for b in budgets]
            plt.plot(budgets, means, marker='s', label='sensitivity', linewidth=2)
            plt.fill_between(budgets, np.array(means)-np.array(stds), np.array(means)+np.array(stds), alpha=0.2)
        
        # Plot passive baseline as horizontal line (lower is better for RMSE/MAE, higher for R2)
        baseline = pas[dataset]['best_metric']
        plt.axhline(baseline, color='k', linestyle='--', label='passive_best', linewidth=2)
        
        plt.xlabel('Labeled budget (max_labels)')
        plt.ylabel(metric)
        plt.title(f'{dataset}: Strategies Comparison ({metric}) - CV + Multiple Trials')
        plt.grid(True, alpha=0.3)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(SAVE_DIR, f'reg_{dataset}_comparison_{metric}.png'), dpi=200, bbox_inches='tight')
        plt.close()
        
        print(f"Saved {dataset} {metric} comparison")



=== Comparing strategies for diabetes ===
Saved diabetes rmse comparison
Saved diabetes mae comparison
Saved diabetes r2 comparison

=== Comparing strategies for linnerud ===
Saved linnerud rmse comparison
Saved linnerud mae comparison
Saved linnerud r2 comparison

=== Comparing strategies for california ===
Saved california rmse comparison
Saved california mae comparison
Saved california r2 comparison


In [3]:
# Create summary table
print("\n=== Summary Table ===")
summary_data = []

for dataset in DATASETS:
    # Get budgets and methods
    budgets = sorted([int(b) for b in next(iter(unc[dataset].values())).keys()])
    methods = list(unc[dataset].keys())
    
    for method in methods + ['sensitivity']:
        if method == 'sensitivity' and dataset not in sen:
            continue
            
        if method == 'sensitivity':
            data = sen[dataset]
        else:
            data = unc[dataset][method]
        
        # Get performance at highest budget
        max_budget = max(budgets)
        rmse_mean = data[str(max_budget)]['rmse_mean']
        rmse_std = data[str(max_budget)]['rmse_std']
        mae_mean = data[str(max_budget)]['mae_mean']
        mae_std = data[str(max_budget)]['mae_std']
        r2_mean = data[str(max_budget)]['r2_mean']
        r2_std = data[str(max_budget)]['r2_std']
        
        summary_data.append({
            'dataset': dataset,
            'method': method,
            'budget': max_budget,
            'rmse_mean': rmse_mean,
            'rmse_std': rmse_std,
            'mae_mean': mae_mean,
            'mae_std': mae_std,
            'r2_mean': r2_mean,
            'r2_std': r2_std
        })

# Convert to DataFrame for nice display
df = pd.DataFrame(summary_data)
print(df.round(4))

# Save summary
df.to_csv(os.path.join(SAVE_DIR, 'reg_comparison_summary.csv'), index=False)
print(f'\nSaved comparison summary to {SAVE_DIR}')
print('All comparison figures and summary saved!')



=== Summary Table ===
       dataset            method  budget  rmse_mean  rmse_std  mae_mean  \
0     diabetes           entropy     200    53.1361    1.0090   42.6384   
1     diabetes            margin     200    53.1361    1.0090   42.6384   
2     diabetes  least_confidence     200    53.1361    1.0090   42.6384   
3     diabetes       sensitivity     200    53.5369    0.9533   43.2594   
4     linnerud           entropy     200    34.1819    3.8368   28.9691   
5     linnerud            margin     200    34.1819    3.8368   28.9691   
6     linnerud  least_confidence     200    34.1819    3.8368   28.9691   
7     linnerud       sensitivity     200    34.1819    3.8368   28.9691   
8   california           entropy     200     1.1526    0.2946    0.7096   
9   california            margin     200     1.1526    0.2946    0.7096   
10  california  least_confidence     200     1.1526    0.2946    0.7096   
11  california       sensitivity     200     1.0039    0.2210    0.6331   

 